<a href="https://colab.research.google.com/github/hsym-uec/myWeb_github/blob/main/Simmple_Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from pandas_datareader import data
import datetime as dt
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import matplotlib.style as style
# import mplfinance as mpf

#日付取得
end_date = dt.date.today()
start_date = end_date + dt.timedelta(weeks=-52)

code = ['9613.JP','9984.JP'] #ntt_data, softbank
name = ['NTT_Data','SoftBank']
# df_ntt = data.DataReader(code, 'stooq', start, end)

#買った時の１株あたりの単価
price = [0,0]
#保有株数
stock = [0,0]

#取得価格＝保有数量x取得単価
price_buy1 = price[0] * stock[0]
price_buy2 = price[1] * stock[1]
price_buy_sum = price_buy1 + price_buy2 #合計

#各株銘柄取得＆終値取得する関数
def get_kb_Close(code, start_date, end_date):
    df = data.DataReader(code,'stooq', start_date, end_date)
    kb_Close = df['Close']
    return kb_Close

DF = pd.DataFrame()

for i in code:
    DF = pd.concat([DF, get_kb_Close(i, start_date, end_date)], axis=1)
DF.columns = name

DF.index = pd.to_datetime(DF.index) #日付データに変換
print(DF)




            NTT_Data  SoftBank
2023-06-26   1977.50   6622.00
2023-06-23   2002.50   6625.00
2023-06-22   2058.50   6787.00
2023-06-21   2072.00   6937.00
2023-06-20   2080.00   6691.00
...              ...       ...
2022-07-05   1922.08   5354.04
2022-07-04   1888.27   5259.46
2022-07-01   1852.48   5108.14
2022-06-30   1865.40   5211.68
2022-06-29   1903.19   5301.28

[244 rows x 2 columns]


In [ ]:
DF['NTT_Data']

In [4]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
from pandas_datareader import data
import datetime as dt
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.style as style

# #各株銘柄取得＆最新(前日)の終値取得する関数
# def get_kb_Close(code):
#     end_date = dt.date.today()
#     start_date = end_date + dt.timedelta(days=-3)
#     stock_code = str(code) + '.JP'
#     df = data.DataReader(stock_code,'stooq', start_date, end_date)
#     kb_Close = df['Close']
#     return kb_Close[0]

#get_kb_Close(9613)

In [ ]:
get_kb_Close(9613)


In [ ]:
def create_server_connection(host_name, user_name, password):
    connection = None
    
    try:
        connection = mysql.connector.connect(
            host = host_name,# ホスト名(IPアドレス）
            user = user_name,
            password = password
		)
        print("MySQL Database connection success")
    except Error as err:
        print(f"Error:'{err}")
    return	connection

In [ ]:
#買った時の価格(1株)を取得する関数
def buy_kb(code, buy_days):
    #表記：'%year-%month-%day
    startdate = buy_days
    enddate = buy_days
    stock_code = str(code) + '.JP'
    df = data.DataReader(stock_code,'stooq', startdate, enddate)
    if df.empty:
        return None
    buy_kb_Close = df['Close']
    return buy_kb_Close[0]

buy_kb(9613,'2023-05-26')

In [ ]:
#取引報告計算(買った場合)
def cal_kb(code, code_name, purchase_num, buy_days):
    connection = None
    passwordy = '507560yosshi'
    
    price = float(buy_kb(code, buy_days)) #買った時の一株あたりの単価
    print(price)
    sum_price = float(price * purchase_num)
    print(sum_price)
    info = 'buy' #売買の判断

    try:
        connection = mysql.connector.connect(
            host = 'localhost',
            user = 'yosshy',
            password = passwordy,
            database = 'stocks_situation'
        )

        cursor = connection.cursor()

        sql = ('''
        insert into transactions
            (stock_code, company_name, trade_situation, trade_day, amount, trade_price, sum_trade_price) 
            values (%s, %s, %s, %s, %s, %s, %s)
        ''')

        val = [
            (code, code_name, info, buy_days, purchase_num, price, sum_price)
        ]
        print(val)

        cursor.executemany(sql, val)
        connection.commit()
        
        print(f"MySQL INSERT success")

        cursor.close()
    except Error as err:
        print(f"Error:' {err}")
    
    finally:
        if connection is not None and connection.is_connected():
            connection.close()

# cal_kb(9613, 'NTT_Data', 100, '2023-05-25')


	

In [ ]:
cal_kb(6758, 'Sony', 100, '2023-05-18')

In [ ]:
cal_kb(7974, 'Nintendo', 100, '2023-05-29')

In [ ]:
#保有株式の現在の状況
def now_kb():
    connection = None
    passwordy = '507560yosshi'

    count = 0
    code_list = []
    company_list = []
    amount_list = []


    try:
        connection = mysql.connector.connect(
            host = 'localhost',
            user = 'yosshy',
            password = passwordy,
            database = 'stocks_situation'
        )

        cursor = connection.cursor(dictionary=True)

        #trade_situationの判定をしない場合(仮)
        get_sql = ('''
            select id, stock_code, company_name, amount from transactions;
        ''')

        insert_sql = ('''
            insert into now_stocks
            (stock_code, company_name, now_price, amount, market_price, now_date)
            values (%s, %s, %s, %s, %s, %s)
        ''')

        cursor.execute(get_sql)
        
        for fetched_line in cursor.fetchall():
            # id = fetched_line['id']
            # print(id)
            stock_code = fetched_line['stock_code']
            company_name = fetched_line['company_name']
            amount = fetched_line['amount']
            #保有株式のコードのリスト
            code_list.append(stock_code)
            #保有株式の名前のリスト
            company_list.append(company_name)
            #保有株式の保有数のリスト
            amount_list.append(amount)
            count += 1

        print(code_list)
        for i in range(count):
            code = int(code_list[i])
            code_name = company_list[i]
            amounts = int(amount_list[i])
            if code is None or amounts is None:
                continue
            now_price = float(get_kb_Close(code))
            # now_price = float(buy_kb(code, '2023-05-26'))
            print(now_price)
            market_price = float(amounts * now_price)
            # now_date = dt.date.today() + dt.timedelta(days=-5)
            now_date = dt.date.today() + dt.timedelta(days=-1)
            val = [(code, code_name, now_price, amounts, market_price, now_date)]
            print(f"{i+1}: MySQL INSERT success")
            print(val)
            cursor.executemany(insert_sql, val)
            connection.commit()
            

        cursor.close()

    except Error as err:
        print(f"Error:' {err}")
    
    finally:
        if connection is not None and connection.is_connected():
            connection.close()

In [ ]:
now_kb()

In [ ]:
now_date = dt.date.today()
print(now_date)

In [ ]:
now_price = float(get_kb_Close(9613))
print(now_price)

### 取引報告計算(売る場合)
	引数：企業コード、売る株数、売る日付
	売る場合→データベースから削除
	売った企業を別のデータベースに保存
	(できたら)利益を保存
* まず、保有株数からデータを持ってくる必要がある
* つまり保有株数を取得する関数を作る→保有株数(trans)
* 各引数に入れる
* 保有株から削除または減らす→まず削除から
* 持っている株以上のデータを要求したらエラー


In [5]:
###保有株数表示
def see_stock():
    connection = None
    passwordy = '507560yosshi'
    now_stock_list = []
    
    try:
        connection = mysql.connector.connect(
            host = 'localhost',
            user = 'yosshy',
            password = passwordy,
            database = 'stocks_situation'
        )

        cursor = connection.cursor(dictionary=True)
        #trade_situationの判定をしない場合(仮)
        get_sql = ('''
            select stock_code, company_name, trade_day, amount from transactions;
        ''')

        cursor.execute(get_sql)
        
        for fetched_line in cursor.fetchall():
            stock_code = fetched_line['stock_code']
            company_name = fetched_line['company_name']
            trade_day = fetched_line['trade_day']
            amount = fetched_line['amount']
            now_stock_list.append([stock_code, company_name, amount, trade_day])
        
        colums = ["企業コード", "企業名", "保有株数", "取引日付"]
        df = pd.DataFrame(data=now_stock_list, columns=colums)
        
        print(now_stock_list)

        cursor.close()

        return df

    except Error as err:
        print(f"Error:' {err}")
    
    finally:
        if connection is not None and connection.is_connected():
            connection.close()

### dataframeの抽出
* df.loc[インデックス指定, カラム指定]

In [6]:

##保有株の変動をグラフで表示する
###まず、保有株の企業コードを取得する
def get_stocks_name(df,name):
    company_code = []
    for i in range(len(df)):
        company_code.append(df.at[i, name])
    return company_code

#各株銘柄取得＆終値取得する関数
def get_kb_Close(code, start_date, end_date):
    df = data.DataReader(code,'stooq', start_date, end_date)
    kb_Close = df['Close']
    return kb_Close

In [23]:
get_kb_Close("9613.JP", "2023-06-12", dt.date.today())

Date
2023-06-26    1977.5
2023-06-23    2002.5
2023-06-22    2058.5
2023-06-21    2072.0
2023-06-20    2080.0
2023-06-19    2123.5
2023-06-16    2135.5
2023-06-15    2108.0
2023-06-14    2134.0
2023-06-13    2068.5
2023-06-12    2059.0
Name: Close, dtype: float64

In [7]:
now_df = see_stock()
company_code = []
company_name = []
company_code = get_stocks_name(now_df, '企業コード') #企業コードが入っているリスト
company_name = get_stocks_name(now_df, '企業名') #企業名が入っているリスト
change_day = get_stocks_name(now_df, '取引日付') #取引日付が入っているリスト
length = len(company_code) #保有している株の数

df_change = pd.DataFrame()
year = dt.date.today() + dt.timedelta(weeks=-52)
for i in range(length):
    df_change = pd.concat([df_change, get_kb_Close(str(company_code[i]) + '.JP', year, dt.date.today())], axis=1)

df_change.columns = company_name
df_change.index = pd.to_datetime(df_change.index) #日付データに変換

print(df_change)

[[9613, 'NTT_Data', 200, datetime.date(2023, 6, 14)], [4901, '富士フイルムＨＬＤＧ', 100, datetime.date(2023, 6, 14)], [7974, '任天堂', 100, datetime.date(2023, 5, 29)], [7832, 'バンダイナムコHLDG', 100, datetime.date(2023, 6, 19)], [6501, '日立', 100, datetime.date(2023, 6, 19)]]
            NTT_Data  富士フイルムＨＬＤＧ   任天堂  バンダイナムコHLDG    日立
2023-06-26   1977.50     8542.00  6289      3267.00  8692
2023-06-23   2002.50     8487.00  6230      3277.00  8700
2023-06-22   2058.50     8634.00  6318      3409.00  8930
2023-06-21   2072.00     8631.00  6212      3431.00  8829
2023-06-20   2080.00     8636.00  6242      3448.00  8747
...              ...         ...   ...          ...   ...
2022-07-05   1922.08     7356.86  5861      9991.20  6493
2022-07-04   1888.27     7383.62  5840      9755.88  6426
2022-07-01   1852.48     7309.29  5736      9613.29  6337
2022-06-30   1865.40     7213.15  5868      9551.47  6438
2022-06-29   1903.19     7221.08  5897      9790.78  6584

[244 rows x 5 columns]


In [24]:
df_change[['NTT_Data','任天堂']]

,NTT_Data,任天堂
2023-06-26,1977.50,6289
2023-06-23,2002.50,6230
2023-06-22,2058.50,6318
2023-06-21,2072.00,6212
2023-06-20,2080.00,6242
...,...,...
2022-07-05,1922.08,5861
2022-07-04,1888.27,5840
2022-07-01,1852.48,5736
2022-06-30,1865.40,5868


In [8]:
import openpyxl

In [10]:
# df_change.to_excel("have_stocks.xlsx")
df_change.to_csv("have_stocks.csv", encoding="shift_jis")
##日本語を含む場合はencoding="shift_jis"が必要

In [ ]:
def sell_stock(code, code_name, purchase_num):
    connection = None
    passwordy = '507560yosshi'
    
    count = 0
    judge = 0
    code_list = []
    company_list = []
    amount_list = []
    trade_day_list = []

    try:
        connection = mysql.connector.connect(
            host = 'localhost',
            user = 'yosshy',
            password = passwordy,
            database = 'stocks_situation'
        )

        cursor = connection.cursor(dictionary=True)
        
        get_sql = ('''
            select stock_code, company_name, trade_day, amount from transactions;
        ''')

        cursor.execute(get_sql)
        
        # この文で取得したテーブルのカラムにあるデータを全て取ってくる
        for fetched_line in cursor.fetchall():
            stock_code = fetched_line['stock_code']
            company_name = fetched_line['company_name']
            trade_day = fetched_line['trade_day']
            amount = fetched_line['amount']
            #保有株式のコードのリスト
            code_list.append(stock_code)
            #保有株式の名前のリスト
            company_list.append(company_name)
            #保有株式の保有数のリスト
            amount_list.append(amount)
            #保有株式を買った日付
            trade_day_list.append(trade_day)
            count += 1
        
        ##一旦、企業コードと企業名と保有株数が一致していたらその株をDBから消す
        delete_sql = ('''
            DELETE FROM transactions where stock_code = %s AND company_name = %s AND amount = %s;
        ''')
        ## 引数に一致する値がデータベースに存在しているかどうか
        ## 引数と一致する値が存在していたらjudgeに１を入れる
        for i in range(count):
            if code == code_list[i] and code_name == company_list[i] and purchase_num == amount_list[i]:
                judge = 1
        
        # judge = 1 ならDBから削除を実行
        if judge == 1:
            val = [(code, code_name, purchase_num)]
            cursor.executemany(delete_sql, val)
            connection.commit()
            print(f"MySQL Delete success")
            
        elif judge == 0:
            print(f"正しい引数を入れてください")
        
        cursor.close()
        return judge

    except Error as err:
        print(f"Error:' {err}")
    
    finally:
        if connection is not None and connection.is_connected():
            connection.close()

In [ ]:
sell_stock(6758, 'Sony', 200)

In [21]:
dif_day = dt.date.today() + dt.timedelta(weeks=-5)
print(dif_day.day)
print(dt.date.today())
print(dt.timedelta(weeks=-5).days)

24
2023-06-28
-35


### code保存
```python:
# スライダーで表示範囲を選択
time_range = st.slider("表示範囲を選択してください", min_value=1, max_value=12, value=1, step=1, format="%d ヶ月")

# 選択された表示範囲に応じてデータを抽出
if time_range == 1:
    selected_df = df_month
elif time_range == 6:
    selected_df = df_sixmonth
else:
    selected_df = df_year

# チェックボックスの状態を取得
for i in range(length):
	company_checkbox.append(col_company[i].checkbox(label=company_name[i]))

# チェックボックスがチェックされている要素のインデックスを取得
checked_indices = [i for i, checkbox in enumerate(company_checkbox) if checkbox]

# チェックボックスがチェックされている要素のデータフレームを抽出
selected_stocks = selected_df.iloc[:, checked_indices]

# 選択された株式の価格推移をグラフで表示
st.line_chart(selected_stocks)
```